In [1]:
import billboard
import pandas as pd

In [2]:
help(billboard)

Help on module billboard:

NAME
    billboard

CLASSES
    builtins.Exception(builtins.BaseException)
        BillboardNotFoundException
        BillboardParseException
    builtins.UserWarning(builtins.Warning)
        UnsupportedYearWarning
    builtins.object
        ChartData
        ChartEntry
            YearEndChartEntry
    
    class BillboardNotFoundException(builtins.Exception)
     |  Method resolution order:
     |      BillboardNotFoundException
     |      builtins.Exception
     |      builtins.BaseException
     |      builtins.object
     |  
     |  Data descriptors defined here:
     |  
     |  __weakref__
     |      list of weak references to the object (if defined)
     |  
     |  ----------------------------------------------------------------------
     |  Methods inherited from builtins.Exception:
     |  
     |  __init__(self, /, *args, **kwargs)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  -------------------

In [3]:
chart = billboard.ChartData('hot-100')

In [4]:
chart.date

'2022-10-22'

In [5]:
chart.entries

[billboard.ChartEntry(title='Bad Habit', artist='Steve Lacy'),
 billboard.ChartEntry(title='Unholy', artist='Sam Smith & Kim Petras'),
 billboard.ChartEntry(title='As It Was', artist='Harry Styles'),
 billboard.ChartEntry(title='I Like You (A Happier Song)', artist='Post Malone Featuring Doja Cat'),
 billboard.ChartEntry(title='You Proof', artist='Morgan Wallen'),
 billboard.ChartEntry(title="I Ain't Worried", artist='OneRepublic'),
 billboard.ChartEntry(title='Sunroof', artist='Nicky Youre & dazy'),
 billboard.ChartEntry(title='Super Freaky Girl', artist='Nicki Minaj'),
 billboard.ChartEntry(title='The Kind Of Love We Make', artist='Luke Combs'),
 billboard.ChartEntry(title='Vegas', artist='Doja Cat'),
 billboard.ChartEntry(title='About Damn Time', artist='Lizzo'),
 billboard.ChartEntry(title='Something In The Orange', artist='Zach Bryan'),
 billboard.ChartEntry(title='Wait For U', artist='Future Featuring Drake & Tems'),
 billboard.ChartEntry(title='Wasted On You', artist='Morgan Wal

In [6]:
chart.entries[3].rank

4

In [7]:
for i in chart:
    print(i)

'Bad Habit' by Steve Lacy
'Unholy' by Sam Smith & Kim Petras
'As It Was' by Harry Styles
'I Like You (A Happier Song)' by Post Malone Featuring Doja Cat
'You Proof' by Morgan Wallen
'I Ain't Worried' by OneRepublic
'Sunroof' by Nicky Youre & dazy
'Super Freaky Girl' by Nicki Minaj
'The Kind Of Love We Make' by Luke Combs
'Vegas' by Doja Cat
'About Damn Time' by Lizzo
'Something In The Orange' by Zach Bryan
'Wait For U' by Future Featuring Drake & Tems
'Wasted On You' by Morgan Wallen
'Titi Me Pregunto' by Bad Bunny
'I'm Good (Blue)' by David Guetta & Bebe Rexha
'Me Porto Bonito' by Bad Bunny & Chencho Corleone
'Late Night Talking' by Harry Styles
'Tomorrow 2' by GloRilla & Cardi B
'She Had Me At Heads Carolina' by Cole Swindell
'Heat Waves' by Glass Animals
'Jimmy Cooks' by Drake Featuring 21 Savage
'Under The Influence' by Chris Brown
'Hold Me Closer' by Elton John & Britney Spears
'5 Foot 9' by Tyler Hubbard
'Thank God' by Kane Brown With Katelyn Brown
'Cuff It' by Beyonce
'Unstoppab

In [8]:
song = chart[0]
help(song)

Help on ChartEntry in module billboard object:

class ChartEntry(builtins.object)
 |  ChartEntry(title, artist, image, peakPos, lastPos, weeks, rank, isNew)
 |  
 |  Represents an entry (typically a single track) on a chart.
 |  
 |  Attributes:
 |      title: The title of the track.
 |      artist: The name of the track artist, as formatted on Billboard.com.
 |          If there are multiple artists and/or featured artists, they will
 |          be included in this string.
 |      image: The URL of the image for the track.
 |      peakPos: The track's peak position on the chart as of the chart date,
 |          as an int (or None if the chart does not include this information).
 |      lastPos: The track's position on the previous week's chart, as an int
 |          (or None if the chart does not include this information).
 |          This value is 0 if the track was not on the previous week's chart.
 |      weeks: The number of weeks the track has been or was on the chart,
 |        

In [9]:
df = pd.read_csv('data/tracks.csv', parse_dates=['release_date'])

In [10]:
# Conseguindo o valor da música mais nova no dataset de features
last_song_date = df['release_date'].max().date()
last_song_date

datetime.date(2021, 4, 16)

In [11]:
df.loc[df['popularity'] == df['popularity'].max()].artists

93802    ['Justin Bieber', 'Daniel Caesar', 'Giveon']
Name: artists, dtype: object

In [36]:
# Identifica o ranking mais próximo da música mais nova
# nc = billboard.ChartData('hot-100', last_song_date)
nc = billboard.ChartData('hot-100', '2022-01-01')
nc

billboard.ChartData('hot-100', date='2022-01-01')

In [37]:
nc.entries

[billboard.ChartEntry(title='All I Want For Christmas Is You', artist='Mariah Carey'),
 billboard.ChartEntry(title="Rockin' Around The Christmas Tree", artist='Brenda Lee'),
 billboard.ChartEntry(title='Jingle Bell Rock', artist='Bobby Helms'),
 billboard.ChartEntry(title='A Holly Jolly Christmas', artist='Burl Ives'),
 billboard.ChartEntry(title='Easy On Me', artist='Adele'),
 billboard.ChartEntry(title="It's The Most Wonderful Time Of The Year", artist='Andy Williams'),
 billboard.ChartEntry(title='Last Christmas', artist='Wham!'),
 billboard.ChartEntry(title='Feliz Navidad', artist='Jose Feliciano'),
 billboard.ChartEntry(title='Stay', artist='The Kid LAROI & Justin Bieber'),
 billboard.ChartEntry(title='Sleigh Ride', artist='The Ronettes'),
 billboard.ChartEntry(title='The Christmas Song (Merry Christmas To You)', artist='Nat King Cole'),
 billboard.ChartEntry(title='Let It Snow, Let It Snow, Let It Snow', artist='Dean Martin'),
 billboard.ChartEntry(title='Underneath The Tree', ar

In [63]:
from datetime import timedelta, datetime, date
import time

In [46]:
week = timedelta(days=7)
week

datetime.timedelta(days=7)

In [47]:
from utils.ranking import rank

In [75]:
# curr_date = last_song_date
curr_date = date.fromisoformat('2022-01-01')
curr_date

datetime.date(2022, 1, 1)

In [76]:
##########
"""
Importante paralelizar o trabalho do scrapper no futuro e pegar mais semanas
"""
##########

start_time = time.time()
chart_entries = []
while curr_date.year >= 2021:
#     print(curr_date)
    nc = billboard.ChartData('hot-100', curr_date)
    
    for track in nc:
        # Tratamento para designar o principal artista
        artist_list = track.artist.split(' ')
        main_artist = artist_list[0]
        if len(artist_list) > 1:
            # Só pegar até o 3 nome para não ficar muito grande
            for name in artist_list[1:3]:
                if name.lower() not in ('featuring', '&', 'and') and '(' not in name:
                    main_artist += ' ' + name
                else:
                    break
        chart_entries.append({'name': track.title, 'artist': main_artist, 'rank': track.rank, 'points': rank(track.rank)})
#     curr_date_str = curr_date.strftime('%d-%m-%Y')
#     entries[curr_date_str] = pd.DataFrame(chart_entries)
    curr_date -= week

# print(chart_entries)
time_diff = time.time() - start_time
print(f'Scrapping time: {time_diff:.2f}s')
entries = pd.DataFrame(chart_entries)

Scrapping time: 66.44s


In [77]:
### Juntar todos os dataframes e calcular os pontos pra poder unir com o dataset de features
entries

,name,artist,rank,points
0,All I Want For Christmas Is You,Mariah Carey,1,10
1,Rockin' Around The Christmas Tree,Brenda Lee,2,10
2,Jingle Bell Rock,Bobby Helms,3,10
3,A Holly Jolly Christmas,Burl Ives,4,10
4,Easy On Me,Adele,5,10
...,...,...,...,...
5295,Hole In The Bottle,Kelsea Ballerini,96,1
5296,"No Body, No Crime",Taylor Swift,97,1
5297,Moonwalking In Calabasas,DDG,98,1
5298,Took Her To The O,King Von,99,1


In [83]:
entries = entries.groupby(['name', 'artist'], as_index=False)['points'].agg('sum')
entries

,name,artist,points
0,'Til You Can't,Cody Johnson,25
1,'Tis The Damn Season,Taylor Swift,1
2,(There's No Place Like) Home For The Holidays,Perry Como,37
3,"1 Step Forward, 3 Steps Back",Olivia Rodrigo,21
4,100.mil',J. Cole,11
...,...,...,...
786,pov,Ariana Grande,80
787,pride.is.the.devil,J. Cole,32
788,punchin'.the.clock,J. Cole,9
789,the.climb.back,J. Cole,9


In [84]:
entries.sort_values('points', ascending=False)

,name,artist,points
373,Levitating,Dua Lipa,474
580,Save Your Tears,The Weeknd,451
276,Heat Waves,Glass Animals,345
352,Kiss Me More,Doja Cat,339
364,Leave The Door Open,Silk Sonic,329
...,...,...,...
680,Tick Tock,Young Thug,1
541,Rare,Nas,1
674,This Bar,Morgan Wallen,1
549,Reminds Me Of You,The Kid LAROI,1


In [80]:
# Coluna de artistas é uma string e não uma lista de strings
df.dtypes

id                          object
name                        object
popularity                   int64
duration_ms                  int64
explicit                     int64
artists                     object
id_artists                  object
release_date        datetime64[ns]
danceability               float64
energy                     float64
key                          int64
loudness                   float64
mode                         int64
speechiness                float64
acousticness               float64
instrumentalness           float64
liveness                   float64
valence                    float64
tempo                      float64
time_signature               int64
artist                      object
dtype: object

In [81]:
from ast import literal_eval
df['artist'] = df['artists'].apply(lambda x: literal_eval(x)[0])

In [82]:
df

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,...,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3,Uli
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,...,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1,Fernando Pessoa
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,...,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5,Ignacio Corsini
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,...,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3,Ignacio Corsini
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922-01-01,0.402,0.1580,...,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4,Dick Haymes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,0.5180,...,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4,阿YueYue
586668,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.6630,...,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4,ROLE MODEL
586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.3140,...,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4,FINNEAS
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",2021-03-05,0.696,0.6150,...,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4,Gentle Bones


In [88]:
merged_df = entries.merge(df, how='inner', on=['name', 'artist'])
merged_df.drop(columns=['id', 'artists', 'id_artists']).drop_duplicates(['name', 'artist'])

,name,artist,points,popularity,duration_ms,explicit,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,2 Much,Justin Bieber,4,82,152796,0,2021-03-19,0.583,0.444,9,-8.601,1,0.0456,0.5930,0.000000,0.3530,0.167,119.590,4
1,7 Summers,Morgan Wallen,43,77,210507,0,2021-01-08,0.466,0.786,0,-5.130,1,0.0328,0.4170,0.000061,0.1250,0.571,203.903,4
3,865,Morgan Wallen,18,76,190680,0,2021-01-08,0.532,0.584,8,-5.254,1,0.0248,0.7230,0.000000,0.1010,0.367,88.003,4
4,Adderall (Corvette Corvette),Popp Hunna,11,76,136224,1,2020-10-15,0.913,0.391,4,-11.158,0,0.3680,0.2720,0.000000,0.0666,0.713,141.050,4
5,Afterglow,Ed Sheeran,56,88,185487,0,2020-12-21,0.641,0.324,11,-5.851,1,0.0299,0.6980,0.000000,0.3280,0.273,110.184,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,Wonder,Shawn Mendes,1,82,172693,0,2020-10-02,0.333,0.637,1,-4.904,0,0.0581,0.1310,0.000018,0.1490,0.132,139.898,4
346,X Gon' Give It To Ya,DMX,6,83,217587,1,2007-06-12,0.761,0.899,10,-3.090,0,0.1830,0.0135,0.000000,0.0719,0.673,95.027,4
347,You Should Probably Leave,Chris Stapleton,91,76,213493,0,2020-11-13,0.602,0.477,9,-8.425,1,0.0320,0.2680,0.000031,0.1730,0.552,183.890,4
348,"You're A Mean One, Mr. Grinch",Thurl Ravenscroft,20,46,315707,0,1966-12-18,0.517,0.484,2,-10.930,0,0.3180,0.9520,0.007500,0.1290,0.420,141.914,3
